In [1]:
from mxnet import nd


def dropout(X, drop_probability):
    keep_probability=1-drop_probability
    assert 0<=keep_probability<=1
    if keep_probability==0:
        return X.zeros_like()
    mask=nd.random.uniform(0,1.0, X.shape,ctx=X.context)<keep_probability
    #设置一个阀值keep_probability，如果小于这个阀值则丢弃
    #numpy.random.uniform(low,high,size)，从一个均匀分布[low,high)中随机采样，注意定义域是左闭右开，即包含low，不包含high.
    scale =  1 / keep_probability#这个的作用是什么？
    return mask * X * scale


In [2]:

A = nd.arange(20).reshape((5,4))
dropout(A, 0.0)




[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]
 [ 16.  17.  18.  19.]]
<NDArray 5x4 @cpu(0)>

In [3]:
#read data-FashionMNIST 
import sys
sys.path.append("..")
import utils
batch_size=256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)


In [4]:
# construct mulitiply perception machine
#real parameters
num_inputs=28*28
num_outputs=10
num_hidden1=256
num_hidden2=256
weight_scale=0.01
learning_rate=0.05
#parameters
w1=nd.random.normal(shape=[num_inputs,num_hidden1],scale=weight_scale)
b1=nd.zeros(num_hidden1)

w2=nd.random.normal(shape=[num_hidden1,num_hidden2],scale=weight_scale)
b2=nd.zeros(num_hidden2)

w3=nd.random.normal(shape=[num_hidden2,num_outputs],scale=weight_scale)
b3=nd.zeros(num_outputs)

params=[w1,b1,w2,b2,w3,b3]

for param in params:
    param.attach_grad()


In [5]:
#define net function 

drop_prob1 = 0.2
drop_prob2 = 0.5


def net(X):
    X = X.reshape((-1, num_inputs))
    # 第一层全连接。
    h1 = nd.relu(nd.dot(X, W1) + b1)
    # 在第一层全连接后添加丢弃层。
    h1 = dropout(h1, drop_prob1)
    # 第二层全连接。
    h2 = nd.relu(nd.dot(h1, W2) + b2)
    # 在第二层全连接后添加丢弃层。
    h2 = dropout(h2, drop_prob2)
    return nd.dot(h2, W3) + b3

In [6]:
#define cost function 
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
from mxnet import autograd
from mxnet import gluon

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

learning_rate = .5

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils.SGD(params, learning_rate/batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))
